In [34]:
!pip install ndjson

In [35]:
import ndjson
import re
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import pandas as pd

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
term_doc_index = {}
with open('/content/drive/MyDrive/merged.txt') as f:
  next(f)
  for line in f:
    word, *docs = line.strip().split(' ')
    term_doc_index[word] = tuple(docs)

In [38]:
doc_id_to_title = {}
shelf_to_title = defaultdict(list)
i=0

f = open('/content/drive/MyDrive/cran_experiment.ndjson')
data_file = ndjson.reader(f)
for doc in data_file:
    title = doc["title"] 
    author = doc["author"]
    id=doc["id"] 
    meta = doc["meta"]
    data = doc["data"]
    i += 1
    doc_id_to_title[int(id)] = title

In [53]:
doc_id_to_title[int(33)]

'the prospects for magneto-aerodynamics .'

In [39]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()
def preprocess_term(term, stem=1):
    term = term.lower()
    #condition of with/without stemming
    return remove_symbols(word) if not stem else remove_symbols(porter.stem(term))
preprocess_term('satisfaction')

'satisfact'

In [63]:
#query method for quering documents
def query(include=[], dont_include=[]):
  all_doc_ids = set()
  for term in include:
    term = preprocess_term(term)
    docs = term_doc_index.get(term, [])
    #if there is atleast one doc in all_doc_ids
    if all_doc_ids:
        #if the doc has ALL THE WORDS
        all_doc_ids.intersection_update(set(docs))
    #if doc has no word
    else:
        all_doc_ids.update(set(docs))
    #if all_doc_ids is empty, stop executing this method
    if not all_doc_ids:
        break
#   all_doc_ids = [int(i.replace('(','').replace(',)','')) for i in all_doc_ids]  
#   print("ALL DOCS:\n", type(list(all_doc_ids)[0]))
  complement_docs = set()
  for term in dont_include:
    term = preprocess_term(term)
    docs = term_doc_index.get(term, [])
    complement_docs.update(set(docs))
    #if there is atleast one doc in complement_docs
    if complement_docs:
        # removing the docs which have all terms which we dont want
        set(all_doc_ids).difference_update(set(docs))
        doc_titles = [doc_id_to_title[int(i)] for i in all_doc_ids]
        # print([(i, type(i)) for i in all_doc_ids if not isinstance(i, str)])
        # print(doc_id_to_title)
    #returning the doc titles as per the query
    # return all_doc_ids
    # break
    return doc_titles

d = query(include=['flow', 'field'], dont_include=['transient'])

In [64]:
d

['the prospects for magneto-aerodynamics .',
 'similitude of hypersonic real-gas flows over slender\nbodies with blunted noses .',
 'a theory of asymmetric hypersonic blunt-body flows .',
 'the law of the wake in the turbulent boundary layer .',
 'the 7 in. x 7 in. hypersonic wind tunnel at r.a.e., farnborough\npart iii - calibration of the flow in the working section .',
 'a method of calculating the short period longitudinal\nstability derivatives of a wing in linearised unsteady\ncompressible flow .',
 'quasi-cylindrical surfaces with prescribed loadings\nin the linearised theory of supersonic flow .',
 'use of local similarity concepts in hypersonic viscous\ninteraction problems .',
 'note on tip-bluntness effects in the supersonic and\nhypersonic regimes .',
 'wakes in axial compressors .',
 'compressibility effects in magneto-aerodynamic flows\npast thin bodies .',
 'the flow field over blunted flat plates and its effect\non turbulent boundary growth and heat transfer at a\nmach 